# Weighted Averages for March Madness




In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [ ]:
# import excel file and add year column and values.
# each excel sheet is imported and put into a dictionary with the key being the respective year
sheets = ['2019','2018','2017','2016','2015','2014']
dat = {}
for s in sheets:
    dat[s] = pd.read_excel('March Madness.xlsx',sheet_name=s)
    dat[s]['Year'] = int(s)
    dat[s] = dat[s].dropna()

In [ ]:
# compress all the data into the same dataframe and delete original dictionary of dataframes
df = dat[sheets[0]]
for s in sheets[1:]:
    df = df.append(dat[s])
df = df.reset_index()
df = df.drop(['index'],axis=1)
del(dat)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# getScore takes a dataframe and returns what the score of that dataframe is based on how it is sorted
# The score is calculated as 10 points for each win of team 1, 9 points for each win of team 2, ...
# 1 point for each win of team 10. If a team is a Cinderella, 5 points are added.
def getScore(data):
    # get the top 10 rows
    data = data.reset_index()
    score = 0
    # calculate the score
    for i in range(0,10):

        # calculate for top 10
        score += (10-i) * data.iloc[i]['Number of Tournament Wins']
        
        # account for cinderella teams
        if data.iloc[i,:]['Cinderella'] > 0:
            score += 5
    # end for
    return score

In [ ]:
# topCorrelations returns the column names of x(int) columns that have the highest R^2 correlation values to column cor
def topCorrelations(x,df,cor):
    top = df.corr()[cor]
    top = top**2
    return list(top.sort_values(ascending = False)[1:x+1].index)

In [ ]:
# scaleData scales the data in columns cols of dataframe df based on the kind of scaling selected
def scaleData(df, cols, kind):
    if kind == "Std":
        for c in cols:
            df[c] = StandardScaler().fit_transform(df[[c]])
    elif kind == "MinMax":
        for c in cols:
            df[c] = MinMaxScaler((0,1)).fit_transform(df[[c]])
    return df

In [ ]:
# weightTests tests each possible combination of coefficients for the weighted averages based on the coefficients in 
# coef_range, storing and returning the results in Dataframe results.
# Dataframe df contains the columns that are going to be weighted, the total number of tournament wins, 
# Cinderella, and a Rank column for the calculated rank of each team.

def weightTests(df, curr_coefs, coef_range, n_columns, results, scale, year):
    # if final coef has been set
    if len(curr_coefs) == n_columns:

        # calculate new rank based on coefficients
        df.loc[:,'Rank'] = 0
        for cl in range(0,n_columns):
            df.loc[:,'Rank'] += df.iloc[:,cl] * curr_coefs[cl]
        # end for

        # calculate new score
        df = df.sort_values(['Rank'],ascending=False)
        endScore = getScore(data = df)

        # create new results entry
        res = {"Score":endScore, 'Year':year, "Scale":scale}
        for cl in range(n_columns):
            res[df.columns[cl]] = curr_coefs[cl]
        #end for

        results = results.append(res, ignore_index = True)
    # end if
    
    # need to set more coefficients
    else:
        for c in coef_range:
            curr_coefs.append(c)
            # call weightTests with new coefficients and same previous values
            results = weightTests(df = df, curr_coefs = curr_coefs,
                        coef_range = coef_range,results = results, 
                        n_columns = n_columns, year = year, scale = scale)
            curr_coefs.pop()
        #end for
    #end else
    
    return results

In [ ]:
# runModel takes a number of correlations to look at (topCorr(int)), which column to correlate to (corVar),
# range of weights (weightRange), a Dataframe (data), and the specific year of data (year).
# runModel gets the top correlations, creates 2 copies of the data and scales them through MinMax and Standard scalers.
# After this, weightTests are run on all 3 copies of the data and stored in the results dataframe that is subsequently
# returned.

def runModel(topCorr, weightRange, data, corVar, year):
    # Get top correlation columns
    cols = topCorrelations(x=topCorr,df=data.drop(['Cinderella'],axis=1),cor=corVar)
    cols.append('Rank')
    print(cols)
    
    # Create results dataframe
    results = pd.DataFrame(columns = cols)
    
    
    # Scale data Std
    df_s = data.copy()
    df_s = scaleData(df = df_s,cols = cols[:-1],kind = "Std")
    # Scale data MinMax
    df_m = data.copy()
    df_m = scaleData(df = df_m, cols = cols[:-1],kind = "MinMax")
    
    cols.append(corVar)
    if 'Cinderella' not in cols:
        cols.append('Cinderella')
        
    data = data[cols]
    df_s = df_s[cols]
    df_m = df_m[cols]
    
    # normal tests
    print("Normal Tests", year)
    results = results.append(weightTests(df = data, curr_coefs = [], coef_range = weightRange, 
                                         results = results, n_columns = topCorr, year = year, scale = "None"))
    # std tests
    print("Std Tests", year)
    results = results.append(weightTests(df = df_s, curr_coefs = [], coef_range = weightRange, 
                                        results = results, n_columns = topCorr, year = year, scale = "Std"))
    # minmax tests
    print("MinMax Tests", year)
    results = results.append(weightTests(df = df_m, curr_coefs = [], coef_range = weightRange, 
                                         results = results, n_columns = topCorr, year = year, scale = "MinMax"))
    return results

In [ ]:
# Model parameters
# weights range
ranges = [0,0.25,0.5,0.75,1]
# number of top features to use for making weighted average
topCorr = 5
# which column to correlate to
corrTo = "Number of Tournament Wins"

In [ ]:
# run tests on each year of data and store in an excel file 
results = pd.DataFrame()
df['Rank'] = 0
# loop through years
for y in df['Year'].unique():
    yr = df[df['Year'] == y]
    results = results.append(runModel(topCorr = topCorr, weightRange = ranges, data = yr.copy(), corVar = corrTo, year = y).copy())

# save results to excel
results.to_excel("Weighted_Results.xlsx")